In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

In [2]:
def calculate_percentages_with_suffix(row, suffix):
    """
    Calculate percentage values for demographic groups dynamically based on a suffix.
    Returns:
        pandas.Series - The row with updated percentage values.
    """
    columns = [f"hispanic_{suffix}", f"white_{suffix}", f"black_{suffix}", f"asian_{suffix}"]
    total = sum(row[col] for col in columns if col in row)
    
    if total > 0:
        for col in columns:
            if col in row:
                row[col] = (row[col] / total)
    return row

### Preprocess 2000 Data

In [3]:
csv_file_path = 'data/race_2000.csv'
df_race_2000 = pd.read_csv(csv_file_path)[['Geography', 'Population Groups', 'Number!!Total population']]

def process_group(fips, group):
    row = {"fips": fips}
    population_groups = group.set_index('Population Groups')['Number!!Total population']
    
    # Hispanic population logic
    if 'Hispanic or Latino (of any race) (200-299)' in population_groups:
        row["hispanic_2000"] = population_groups['Hispanic or Latino (of any race) (200-299)']
    else:
        row["hispanic_2000"] = (
            population_groups.get('White alone', 0) - population_groups.get('White alone, not Hispanic or Latino', 0)
            if 'White alone' in population_groups and 'White alone, not Hispanic or Latino' in population_groups
            else 0
        )
    
    # Assign other population groups
    row["white_2000"] = population_groups.get('White alone, not Hispanic or Latino', 
                     population_groups.get('White alone', 0))
    row["black_2000"] = population_groups.get('Black or African American alone', 0)
    row["asian_2000"] = population_groups.get('Asian alone (400-499)', 0)
    row["total_2000"] = population_groups.get('Total population', 0)
    return row

df_race_2000 = pd.DataFrame(
    process_group(fips, group)
    for fips, group in df_race_2000.groupby('Geography')
)

df_race_2000['fips'] = df_race_2000['fips'].str.replace('1600000US', '')
df_race_2000.rename(columns={'Geography': 'fips'}, inplace=True)
df_race_2000 = df_race_2000.apply(calculate_percentages_with_suffix, axis=1, suffix="2000")

# Change wrong values
df_race_2000.loc[df_race_2000['fips'] == "1319007", 'fips'] = "1319000"

In [4]:
display(df_race_2000.head())

,fips,hispanic_2000,white_2000,black_2000,asian_2000,total_2000
0,0100124,0.021922,0.586847,0.391231,0.0,3000
1,0100460,0.000000,0.792537,0.207463,0.0,4774
2,0100484,0.000000,1.000000,0.000000,0.0,714
3,0100676,0.000000,0.000000,1.000000,0.0,543
4,0100820,0.021200,0.872885,0.105914,0.0,23210


### Preprocess 2010 Data

In [5]:
csv_file_path = 'data/race_2010.csv'
df_2010 = pd.read_csv(csv_file_path)

# Remove unwanted artefacts
df_2010['Total'] = df_2010['Total'].str.replace(r'\(.*?\)', '', regex=True)

columns_to_extract = {
    "Geography": "fips",
    "Total": "total_2010",
    "Total!!Hispanic or Latino": "hispanic_2010",
    "Total!!Not Hispanic or Latino!!Population of one race!!White alone": "white_2010",
    "Total!!Not Hispanic or Latino!!Population of one race!!Black or African American alone": "black_2010",
    "Total!!Not Hispanic or Latino!!Population of one race!!Asian alone": "asian_2010",
}

df_race_2010 = df_2010[list(columns_to_extract.keys())].rename(columns=columns_to_extract)
df_race_2010['fips'] = df_race_2010['fips'].str.replace('1600000US', '')
df_race_2010 = df_race_2010.apply(calculate_percentages_with_suffix, axis=1, suffix="2010")

# Change wrong values
df_race_2010.loc[df_race_2010['fips'] == "2127982", 'fips'] = "2148000"

In [6]:
display(df_race_2010.head())

,fips,total_2010,hispanic_2010,white_2010,black_2010,asian_2010
0,0100100,192,0.015789,0.678947,0.305263,0.000000
1,0100124,2688,0.030873,0.543298,0.417922,0.007907
2,0100460,4522,0.022855,0.520726,0.453731,0.002689
3,0100484,758,0.003968,0.993386,0.001323,0.001323
4,0100676,356,0.002817,0.129577,0.867606,0.000000


### Preprocess 2020 Data

In [7]:
csv_file_path = 'data/race_2020.csv'
df_2020 = pd.read_csv(csv_file_path)

columns_to_extract = {
    "Geography": "fips",
    " !!Total:": "total_2020",
    " !!Total:!!Hispanic or Latino": "hispanic_2020",
    " !!Total:!!Not Hispanic or Latino:!!Population of one race:!!White alone": "white_2020",
    " !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Black or African American alone": "black_2020",
    " !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Asian alone": "asian_2020",
}

df_race_2020 = df_2020[list(columns_to_extract.keys())].rename(columns=columns_to_extract)
df_race_2020['fips'] = df_race_2020['fips'].str.replace('1600000US', '')
df_race_2020 = df_race_2020.apply(calculate_percentages_with_suffix, axis=1, suffix="2020")

In [8]:
display(df_race_2020.head())

,fips,total_2020,hispanic_2020,white_2020,black_2020,asian_2020
0,0100100,133,0.030075,0.714286,0.255639,0.000000
1,0100124,2358,0.029476,0.509019,0.454905,0.006599
2,0100460,4366,0.043169,0.408918,0.545541,0.002372
3,0100484,659,0.012638,0.984202,0.000000,0.003160
4,0100676,225,0.004525,0.085973,0.900452,0.009050


In [9]:
import pandas as pd

def adjust_and_normalize_columns(df, columns, threshold=0.00001):
    """
    Combines setting small values to zero, ensuring values are non-negative, 
    and normalizing the specified columns to sum to 1.
    Returns:
        pd.DataFrame: The modified DataFrame with adjusted and normalized columns.
    """
    if not all(column in df.columns for column in columns):
        raise ValueError("Some specified columns are not present in the DataFrame.")

    for column in columns:
        df[column] = df[column].apply(lambda x: max(0, x))
        df[column] = df[column].apply(lambda x: 0 if x < threshold else x)

    # Normalize rows where the sum is greater than zero
    column_sums = df[columns].sum(axis=1)
    df[columns] = df[columns].div(column_sums.where(column_sums > 0, 1), axis=0)

    return df


In [10]:
column_patterns = {
    'df_race_2000': ['hispanic_2000', 'white_2000', 'black_2000', 'asian_2000'],
    'df_race_2010': ['hispanic_2010', 'white_2010', 'black_2010', 'asian_2010'],
    'df_race_2020': ['hispanic_2020', 'white_2020', 'black_2020', 'asian_2020']
}

# df_race_2000, df_race_2010, df_race_2020 should be defined
for df_name, columns in column_patterns.items():
    df = globals()[df_name]
    globals()[df_name] = adjust_and_normalize_columns(df, columns)

In [11]:
# Merge race datasets for easy lookup
columns_to_estimate = ['hispanic', 'white', 'black', 'asian']

race_data = df_race_2000[['fips', 'total_2000'] + [f'{col}_2000' for col in columns_to_estimate]].merge(
    df_race_2010[['fips', 'total_2010'] + [f'{col}_2010' for col in columns_to_estimate]], on='fips'
).merge(
    df_race_2020[['fips', 'total_2020'] + [f'{col}_2020' for col in columns_to_estimate]], on='fips'
)

# Convert the 'fips' column to integers
race_data['fips'] = race_data['fips'].astype(int)
race_data = race_data.reset_index(drop=True)

In [12]:
def estimate_values_for_dataframe(df, race_data, columns):
    """
    Estimate values for each row in a DataFrame based on given year and fips.
    Returns:
        pd.DataFrame: Original DataFrame with estimated values merged.
    """

    def predict_linear_for_year(y_observed, x_observed, target_year):
        """
        Predict the value for a given year using linear fitting.
        """
        def linear_model(x, a, b):
            return a * x + b

        params, _ = curve_fit(linear_model, x_observed, y_observed)
        predicted_value = linear_model(target_year, *params)
        return predicted_value

    observed_years = np.array([2000, 2010, 2020])
    estimated_values = []

    for _, row in df.iterrows():
        year = row['year']
        fips = row['fips']
        race_row = race_data[race_data['fips'] == fips]

        if race_row.empty:
            estimated_values.append({col: np.nan for col in columns})
            continue

        race_row = race_row.iloc[0]

        estimates = {}
        for col in columns:
            y_observed = np.array([
                race_row[f'{col}_2000'],
                race_row[f'{col}_2010'],
                race_row[f'{col}_2020']
            ])
            estimates[col] = predict_linear_for_year(y_observed, observed_years, year)

        estimated_values.append(estimates)

    estimated_df = pd.DataFrame(estimated_values)
    result_df = pd.concat([df.reset_index(drop=True), estimated_df], axis=1)

    return result_df


In [13]:
columns_to_estimate = ['total', 'hispanic', 'white', 'black', 'asian']

df_mayor = pd.read_csv('data/data_mayoral.csv')
df_mayor = estimate_values_for_dataframe(df_mayor, race_data, columns_to_estimate)
df_mayor['total'] = df_mayor['total'].astype(int)

In [14]:
columns_to_normalize = ['hispanic', 'white', 'black', 'asian']
df_mayor = adjust_and_normalize_columns(df_mayor, columns_to_normalize)

In [15]:
df_mayor.head()

,state_abb,geo_name,year,contest,fips,full_name_rep,vote_share_rep,incumbent_rep,contributor.cfscore_rep,prob_democrat_rep,...,gender_est_dem,race_est_dem,pid_est_dem,percent_women,pres_pctD,total,hispanic,white,black,asian
0,DE,wilmington,1992,151000_2019_10_montgomery_AL_Mayor_mayor_1,1077580,beatrice patton carroll,0.090000,0.0,1.011667,0.0,...,M,non_white,D,0.525989,0.891903,73060,0.086214,0.337033,0.570731,0.006021
1,DE,wilmington,1996,2148000_1998_11_louisville_KY_Mayor_mayor_1,1077580,bradley zuber,0.390000,0.0,0.927143,0.0,...,M,non_white,D,0.525989,0.890173,72707,0.094095,0.327885,0.571025,0.006995
2,DE,wilmington,2004,2205000_2016_12_baton rouge_LA_Mayor_mayor_1,1077580,robert bovell,0.270000,0.0,0.776190,0.0,...,M,white,D,0.525989,0.886721,72000,0.109856,0.309588,0.571614,0.008942
3,DE,wilmington,2016,1263000_2021_11_st. petersburg_FL_Mayor_mayor_1,1077580,robert martin,0.118280,0.0,1.010000,0.0,...,M,white,D,0.525989,0.878289,70941,0.133499,0.282142,0.572496,0.011863
4,DC,washington,1990,2404000_2020_11_baltimore_MD_Mayor_mayor_1,1150000,maurice turner,0.113402,0.0,0.843333,0.0,...,F,non_white,D,0.525549,0.914489,503623,0.058937,0.229147,0.698828,0.013088


In [16]:
df_mayor.to_csv('data/data_mayoral.csv', index=False)